# Imports

In [ ]:
import os
import re

import keras
import keras_hub
import nltk
import numpy as np
import pandas as pd
import string
import tensorflow as tf


from collections import defaultdict

from dataclasses import dataclass

from keras import layers
from keras.layers import TextVectorization

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from pprint import pprint
from sklearn.preprocessing import LabelEncoder

# Mount Google

In [ ]:
## Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Exploration

O dataset escolhido está no link a seguir: https://www.kaggle.com/datasets/jessicali9530/kuc-hackathon-winter-2018

É basicamente um dataset que possui dados de reviews de medicamentos. Possui a review em si, o nome do medicamento daquela review, o que aquele medicamento trata, a nota dada na review e algumas outras informações

In [ ]:
!curl -L -o /content/kuc-hackathon-winter-2018.zip\
  https://www.kaggle.com/api/v1/datasets/download/jessicali9530/kuc-hackathon-winter-2018

!unzip -q /content/kuc-hackathon-winter-2018.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 40.6M  100 40.6M    0     0  51.1M      0 --:--:-- --:--:-- --:--:-- 51.1M


In [ ]:
dataset_train = pd.read_csv('/content/drugsComTrain_raw.csv')
dataset_test = pd.read_csv('/content/drugsComTest_raw.csv')

len(dataset_train), len(dataset_test)

(161297, 53766)

In [ ]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161297 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     161297 non-null  int64 
 1   drugName     161297 non-null  object
 2   condition    160398 non-null  object
 3   review       161297 non-null  object
 4   rating       161297 non-null  int64 
 5   date         161297 non-null  object
 6   usefulCount  161297 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 8.6+ MB


In [ ]:
dataset_train = dataset_train[~dataset_train['condition'].isna()] ## Removendo instâncias com dados nulos
dataset_test = dataset_test[~dataset_test['condition'].isna()]

len(dataset_train), len(dataset_test)

(160398, 53471)

Nossa ideia é usar a coluna condition como supervisão de um problema de classificação. As outras informações fora a review e condition não serão utilizadas no treinamento

In [ ]:
dataset_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [ ]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 160398 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     160398 non-null  int64 
 1   drugName     160398 non-null  object
 2   condition    160398 non-null  object
 3   review       160398 non-null  object
 4   rating       160398 non-null  int64 
 5   date         160398 non-null  object
 6   usefulCount  160398 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 9.8+ MB


Como o dataset é muito grande e possui diversas conditions, vamos utilizar um subset do dataset. No final, teremos 44 classes para classificar as reviews

As instâncias de classes fora as 44 mais recorrentes no dataset foram removidas

In [ ]:
dataset_train['condition'].value_counts()


,count
condition,
Birth Control,28788
Depression,9069
Pain,6145
Anxiety,5904
Acne,5588
...,...
26</span> users found this comment helpful.,1
Patent Ductus Arteriosus,1
Scleroderma,1


In [ ]:
train_counts = dataset_train['condition'].value_counts()
train_counts = train_counts[train_counts.values > 615] ## Pain * 0.2

In [ ]:
dataset_train = dataset_train[dataset_train['condition'].isin(train_counts.index)]
dataset_test = dataset_test[dataset_test['condition'].isin(train_counts.index)]

In [ ]:
dataset_train['condition'].value_counts().head()

,count
condition,
Birth Control,28788
Depression,9069
Pain,6145
Anxiety,5904
Acne,5588


In [ ]:
dataset_train['condition'].unique()

array(['ADHD', 'Birth Control', 'Opiate Dependence',
       'Emergency Contraception', 'Bipolar Disorde',
       'Migraine Prevention', 'Depression', 'Cough', 'Obesity',
       'Urinary Tract Infection', 'ibromyalgia', 'Insomnia',
       'Rheumatoid Arthritis', 'Vaginal Yeast Infection', 'Panic Disorde',
       'Migraine', 'Pain', 'Irritable Bowel Syndrome', 'Osteoarthritis',
       'Constipation', 'Bowel Preparation', 'Muscle Spasm', 'Hepatitis C',
       'Overactive Bladde', 'Diabetes, Type 2', 'Smoking Cessation',
       'Anxiety', 'Acne', 'Erectile Dysfunction', 'Chronic Pain',
       'Major Depressive Disorde', 'Anxiety and Stress',
       'High Blood Pressure', 'Allergic Rhinitis',
       'Abnormal Uterine Bleeding', 'Weight Loss',
       'Generalized Anxiety Disorde', 'Back Pain', 'Bacterial Infection',
       'Sinusitis', 'GERD', 'Multiple Sclerosis', 'Nausea/Vomiting',
       'Hyperhidrosis'], dtype=object)

In [ ]:
dataset_train['condition'].nunique()

44

In [ ]:
len(dataset_train), len(dataset_test)

(119595, 40028)

In [ ]:
dataset_train.drop(columns=['uniqueID', 'drugName', 'rating', 'date', 'usefulCount'], inplace=True)
dataset_test.drop(columns=['uniqueID', 'drugName', 'rating', 'date', 'usefulCount'], inplace=True)

In [ ]:
dataset_train = dataset_train[['review', 'condition']]
dataset_test = dataset_test[['review', 'condition']]

O dataset já vem dividido em treino e teste

In [ ]:
dataset_train.head()

,review,condition
1,"""My son is halfway through his fourth week of ...",ADHD
2,"""I used to take another oral contraceptive, wh...",Birth Control
3,"""This is my first time using any form of birth...",Birth Control
4,"""Suboxone has completely turned my life around...",Opiate Dependence
6,"""He pulled out, but he cummed a bit in me. I t...",Emergency Contraception


In [ ]:
dataset_test.head()

,review,condition
0,"""I&#039;ve tried a few antidepressants over th...",Depression
2,"""Quick reduction of symptoms""",Urinary Tract Infection
3,"""Contrave combines drugs that were used for al...",Weight Loss
4,"""I have been on this birth control for one cyc...",Birth Control
6,"""I&#039;ve had the copper coil for about 3 mon...",Birth Control


In [ ]:
dataset = pd.concat([dataset_train, dataset_test], ignore_index=True)

In [ ]:
dataset.head()

,review,condition
0,"""My son is halfway through his fourth week of ...",ADHD
1,"""I used to take another oral contraceptive, wh...",Birth Control
2,"""This is my first time using any form of birth...",Birth Control
3,"""Suboxone has completely turned my life around...",Opiate Dependence
4,"""He pulled out, but he cummed a bit in me. I t...",Emergency Contraception


In [ ]:
len(dataset), len(dataset_train), len(dataset_test)

(159623, 119595, 40028)

### 10 exemplos de instâncias no dataset de treino

In [ ]:
for index, row in dataset_train.iterrows():
  if index == 10:
    break

  review = row['review']
  condition = row['condition']

  print(f" ==== Instância - {index} ====")

  print("Review: ", review)
  print()
  print("Condição: ", condition)
  print()

##&#039;


 ==== Instância - 1 ====
Review:  "My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. 
We have tried many different medications and so far this is the most effective."

Condição:  ADHD

 ==== Instância - 2 ====
Review:  "I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gest

## Modelo usando a Assumption de Markov e Bigramas

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def preprocess(text):
    text = text.lower()
    tokens = word_tokenize(text) #Split das palavras
    tokens = [t for t in tokens if t not in string.punctuation] #Remove pontuações
    tokens = [t for t in tokens if t not in stopwords.words('english')] #Remove as stopwrods
    tokens = ['<s>'] + tokens + ['</s>']  # Marcadores de início/fim
    return tokens

In [ ]:
class MarkovClassifier:
    """
    Ideia é que a classe vai aprender Bi-Gramas específicos mais recorrentes de cada uma das 44 condições
    """
    def __init__(self, classes, alpha=1.0):
        self.classes = classes
        self.alpha = alpha  # Suavização de Laplace
        self.class_stats = {
            c: {
                'bigram_counts': defaultdict(int), # Dicionário que indica a quantidade de ocorrências de cada bigrama
                'prev_counts': defaultdict(int),
                'vocab': set(), # Vocabulário presente para cada classe
                'prior': 0.0
            } for c in classes
        }

    def train(self, X_train, y_train):
        # Calcular priors

        total = len(y_train)
        class_counts = defaultdict(int)
        for c in y_train:
            class_counts[c] += 1 # Basicamente calcula a quantidade de instâncias de cada classe no dataset

        for c in self.classes:
            print(c)

            # Coletar dados da classe
            class_data = [text for text, label in zip(X_train, y_train) if label == c]
            self.class_stats[c]['prior'] = class_counts[c] / total # Quantidade de instâncias da classe pelo total

            # Construir contagens de bigramas
            vocab = set()
            for text in class_data: #Para cada texto de cada classe
                tokens = preprocess(text)
                vocab.update(tokens) #
                for i in range(len(tokens)-1):
                    prev, curr = tokens[i], tokens[i+1]
                    self.class_stats[c]['bigram_counts'][(prev, curr)] += 1 #Conta as ocorrências do bigrama para aquela classe
                    self.class_stats[c]['prev_counts'][prev] += 1 #Conta a ocorrência da primeira palavra do bigrama para a alcasse

            self.class_stats[c]['vocab'] = vocab #Preenche o vocabulário

    def _calculate_log_prob(self, text, target_class):
        tokens = preprocess(text)
        log_prob = np.log(self.class_stats[target_class]['prior'])
        V = len(self.class_stats[target_class]['vocab'])

        for i in range(len(tokens)-1):
            prev, curr = tokens[i], tokens[i+1]
            count_bigram = self.class_stats[target_class]['bigram_counts'].get((prev, curr), 0)
            count_prev = self.class_stats[target_class]['prev_counts'].get(prev, 0)

            # Suavização de Laplace
            prob = (count_bigram + self.alpha) / (count_prev + self.alpha * V)
            log_prob += np.log(max(prob, 1e-10))  # Evitar zeros

        return log_prob

    def predict(self, text):
        max_log_prob = -np.inf
        best_class = None
        for c in self.classes:
            log_prob = self._calculate_log_prob(text, c)
            if log_prob > max_log_prob: #Acha a classe com maior probabilidade de ser
                max_log_prob = log_prob
                best_class = c
        return best_class

    def evaluate(self, X_test, y_test):
        correct = 0
        total = len(X_test)
        contador = 0
        for text, true_label in zip(X_test, y_test):
            predicted = self.predict(text)
            if predicted == true_label:
                correct += 1
            contador += 1
            if contador % 100 == 0:
                print(f"Progresso: {contador}/{total}")
        return correct / total


In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(dataset_train.condition.values)
y_test = label_encoder.transform(dataset_test.condition.values)

In [ ]:
classifier = MarkovClassifier(classes=np.unique(y_train))
classifier.train(dataset_train.review.values, y_train)

accuracy = classifier.evaluate(dataset_test.review.values, y_test)
print(f"Acurácia do modelo: {accuracy:.2f}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
Progresso: 100/40028
Progresso: 200/40028
Progresso: 300/40028
Progresso: 400/40028
Progresso: 500/40028
Progresso: 600/40028
Progresso: 700/40028
Progresso: 800/40028
Progresso: 900/40028
Progresso: 1000/40028
Progresso: 1100/40028
Progresso: 1200/40028
Progresso: 1300/40028
Progresso: 1400/40028
Progresso: 1500/40028
Progresso: 1600/40028
Progresso: 1700/40028
Progresso: 1800/40028
Progresso: 1900/40028
Progresso: 2000/40028
Progresso: 2100/40028
Progresso: 2200/40028
Progresso: 2300/40028
Progresso: 2400/40028
Progresso: 2500/40028
Progresso: 2600/40028
Progresso: 2700/40028
Progresso: 2800/40028
Progresso: 2900/40028
Progresso: 3000/40028
Progresso: 3100/40028
Progresso: 3200/40028
Progresso: 3300/40028
Progresso: 3400/40028
Progresso: 3500/40028
Progresso: 3600/40028
Progresso: 3700/40028
Progresso: 3800/40028
Progresso: 3900/40028
Progresso: 4000/40028
Progres